In [30]:
import json
record_path = "/dataset/zhuotaotian/bhpeng/vectordb/output-group-1-nomask/record.json"
index_range = {
        "ag": [
            0,
            24
        ],
        "amazon": [
            24,
            48
        ],
        "dbpedia": [
            48,
            72
        ],
        "yahoo": [
            72,
            96
        ],
        "yelp": [
            96,
            120
        ]
    }
tasks = ['ag', 'amazon', 'dbpedia', 'yahoo', 'yelp']
topk = 12
groups = 1

for k, v in index_range.items():
    index_range[k] = list(range(v[0], v[1]))

In [33]:
with open(record_path) as f:
    data = json.load(f)
print(data['ag'][0], len(data['ag']))

{'group': 0, 'index': 19, 'label': 'ag'} 91200


In [38]:
def compute_accuracy(data, thresh=0.5):
    accuracy = dict()
    for task in tasks:
        accuracy[task] = dict(total_num=0, total_right=0, single_right_rate=0)
        single_right = 0
        for i, d in enumerate(data[task]):
            accuracy[task]['total_num'] += 1
            if d['index'] in index_range[task]:
                accuracy[task]['total_right'] += 1
                single_right += 1
            if i % (topk*groups) == (topk * groups - 1):
                if single_right / topk / groups > thresh:
                    accuracy[task]['single_right_rate'] += 1
                single_right = 0
        accuracy[task]['total_accuracy'] = accuracy[task]['total_right'] / accuracy[task]['total_num']
        accuracy[task]['single_right_rate'] /= (len(data[task]) / groups / topk)
    return accuracy
accuracy = compute_accuracy(data)
for attr in ['total_accuracy', 'single_right_rate']:
    print(attr+":")
    s=0
    for task in tasks:
        print("  ",  "{:7s}: ".format(task), accuracy[task][attr])
        s+=float(accuracy[task][attr])
    print("  ",  "{:7s}: ".format('avg'), s/5)

total_accuracy:
   ag     :  0.905921052631579
   amazon :  0.9276206140350877
   dbpedia:  0.9095394736842105
   yahoo  :  0.7856688596491228
   yelp   :  0.9472149122807018
   avg    :  0.8951929824561404
single_right_rate:
   ag     :  0.915921052631579
   amazon :  0.9222368421052631
   dbpedia:  0.9106578947368421
   yahoo  :  0.7885526315789474
   yelp   :  0.9397368421052632
   avg    :  0.8954210526315791
